In [1]:
from dask.distributed import Client, LocalCluster, worker, progress, wait
import dask
from dask_jobqueue import SLURMCluster

In [2]:
cluster = SLURMCluster()
client=Client(cluster)
cluster.scale(20)
client


Bad key "backend.qt5" on line 5 in
/home/mrayson/.config/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


Client Scheduler: tcp://146.118.38.109:39457 Dashboard: http://146.118.38.109:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime
import numpy as np
from os import system as sys

import xarray as xr
import pandas as pd
import dask
import glob
from scipy import signal

import matplotlib.pyplot as plt
import matplotlib as mpl

In [4]:
cluster = SLURMCluster()

/home/mrayson/group/mrayson/miniconda3/envs/soda/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [7]:
basedir = '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/'
roms = xr.open_mfdataset('{}/fwd_ver_7*.nc'.format(basedir),
                         concat_dim='ocean_time',
                         parallel=True)

In [22]:
def write_filtered_var(myzeta, varname, is2d, klayer):    
    for ii,t1 in enumerate(times[0:-1]):
        daysince2000 = (t1-np.datetime64('2000-01-01')).astype(float)*1e-9/86400
        outfile = '../tmp/std_i_{}.nc'.format(int(daysince2000))
        print('\tSaving {} for step {}'.format(varname,t1))
        # This is way quicker when working off the chunked zarr data
        zeta_f_time = myzeta.sel(ocean_time = slice(t1-np.timedelta64(1,'D'), t1+np.timedelta64(5,'D'))).std(axis=0, keepdims=True)
        nc = Dataset(outfile, mode='r+')
        #nc['ocean_time'][0] = daysince2000
        #nc['zeta'][:] = zeta_fp_std[ii,...]
        # 2D variable
        if is2d:
            nc[varname][:] = zeta_f_time
        else:
            # 3D variable
            nc[varname][0,klayer,...] = zeta_f_time
        nc.close()

def filter_roms_2d(roms, varname, cutoff_dt, dt):    
    timevar, yvar, xvar = roms[varname].dims
    arr = roms[varname].chunk({'ocean_time':-1,yvar:10,xvar:10})
    # Compute the filted variable
    arr_filt = xr.apply_ufunc(filt,
                              arr, cutoff_dt, dt,
                              dask='parallelized',
                              #dask='allowed',
                              output_dtypes=[arr.dtype],
                              kwargs={'axis': 0, 'order':8}
                             )
    return arr_filt

def filter_roms_3d(roms, varname, klayer, cutoff_dt, dt):    
    timevar,zvar, yvar, xvar = roms[varname].dims
    # Subset by layer
    s_rho = roms['s_rho'].values
    myroms = roms.sel(s_rho=s_rho[klayer])    
    arr = myroms[varname].chunk({'ocean_time':-1,yvar:10,xvar:10})
    # Compute the filted variable
    arr_filt = xr.apply_ufunc(filt,
                              arr, cutoff_dt, dt,
                              dask='parallelized',
                              #dask='allowed',
                              output_dtypes=[arr.dtype],
                              kwargs={'axis': 0, 'order':8}
                             )
    return arr_filt	

def filt(ytmp, cutoff_dt, dt, btype='low', order=3, axis=0 ):
    """
    Butterworth filter the time series
    Inputs:
        cutoff_dt - cuttoff period [seconds]
        btype - 'low' or 'high' or 'band'
    """
    if not btype == 'band':
        Wn = dt/cutoff_dt
    else:
        Wn = [dt/co for co in cutoff_dt]    
    #(b, a) = signal.butter(order, Wn, btype=btype, analog=0, output='ba', fs=1)    
    #return signal.filtfilt(b, a, ytmp, axis=axis, padlen=0)
    print(ytmp.shape, dt, cutoff_dt)
    sos = signal.butter(order, Wn, btype=btype, analog=0, output='sos', fs=1)    
    return signal.sosfiltfilt(sos, ytmp, axis=axis, padlen=0)	


In [9]:
myroms = roms.sel(ocean_time=slice('2020-10-05','2020-12-04'))
varnames = ['zeta','ubar','vbar','temp','salt','u','v']
is2ds = [True, True, True, False, False, False, False]
klayers = 30
times = np.array(pd.date_range('2020-10-09','2020-12-04', freq='4D'))
cutoff_dt = 34*3600.0
# start of the processing
# Save to "std" format file
# Create a file from template
infile = '{}/std_i_template.nc'.format(basedir)
dsin = xr.open_dataset(infile)

In [12]:
!mkdir ../tmp

In [14]:
for t1 in times[0:-1]:
    daysince2000 = (t1-np.datetime64('2000-01-01'))/np.timedelta64(1, 's')/86400
    outfile = '../tmp/std_i_{}.nc'.format(int(daysince2000))
    dsin.to_netcdf(outfile)
    print(outfile)


../tmp/std_i_7587.nc
../tmp/std_i_7591.nc
../tmp/std_i_7595.nc
../tmp/std_i_7599.nc
../tmp/std_i_7603.nc
../tmp/std_i_7607.nc
../tmp/std_i_7611.nc
../tmp/std_i_7615.nc
../tmp/std_i_7619.nc
../tmp/std_i_7623.nc
../tmp/std_i_7627.nc
../tmp/std_i_7631.nc
../tmp/std_i_7635.nc
../tmp/std_i_7639.nc


In [15]:
myroms = roms.sel(ocean_time=slice('2020-10-05','2020-12-04'))

In [16]:
dt = myroms.ocean_time[2] - myroms.ocean_time[1]
dtsec = dt.values.astype(float)*1e-9    

In [17]:
dt, dtsec

(<xarray.DataArray 'ocean_time' ()>
 array(7200000000000, dtype='timedelta64[ns]'), 7200.0)

In [18]:
zeta_f = filter_roms_2d(myroms, 'zeta', cutoff_dt, dtsec)

(0, 0, 0) 7200.0 122400.0


In [19]:
timevar, yvar, xvar = zeta_f.dims
# Rechunk the data
zeta_fpc = zeta_f.chunk({'ocean_time':1,yvar:-1,xvar:-1})
zeta_fpc = zeta_fpc.persist()        

In [20]:
times[0]

numpy.datetime64('2020-10-09T00:00:00.000000000')

In [23]:
write_filtered_var(zeta_fpc, 'zeta', True, 0)

	Saving zeta for step 2020-10-09T00:00:00.000000000
	Saving zeta for step 2020-10-13T00:00:00.000000000
	Saving zeta for step 2020-10-17T00:00:00.000000000
	Saving zeta for step 2020-10-21T00:00:00.000000000
	Saving zeta for step 2020-10-25T00:00:00.000000000
	Saving zeta for step 2020-10-29T00:00:00.000000000
	Saving zeta for step 2020-11-02T00:00:00.000000000
	Saving zeta for step 2020-11-06T00:00:00.000000000
	Saving zeta for step 2020-11-10T00:00:00.000000000
	Saving zeta for step 2020-11-14T00:00:00.000000000
	Saving zeta for step 2020-11-18T00:00:00.000000000
	Saving zeta for step 2020-11-22T00:00:00.000000000
	Saving zeta for step 2020-11-26T00:00:00.000000000
	Saving zeta for step 2020-11-30T00:00:00.000000000


In [69]:
t1=times[0]
zeta_f_time = zeta_fpc.sel(ocean_time = slice(t1-np.timedelta64(1,'D'), t1+np.timedelta64(5,'D')))

In [70]:
zeta_f_time

<xarray.DataArray 'zeta' (ocean_time: 75, eta_rho: 482, xi_rho: 1010)>
dask.array<shape=(75, 482, 1010), dtype=float32, chunksize=(1, 482, 1010)>
Coordinates:
    lon_rho     (eta_rho, xi_rho) float64 dask.array<shape=(482, 1010), chunksize=(482, 1010)>
    lat_rho     (eta_rho, xi_rho) float64 dask.array<shape=(482, 1010), chunksize=(482, 1010)>
  * ocean_time  (ocean_time) datetime64[ns] 2020-10-08 2020-10-08T02:00:00 ...
Dimensions without coordinates: eta_rho, xi_rho

In [74]:
temp[0,:,:] = zeta_f_time.std(dim='ocean_time')

IndexError: too many indices

In [72]:
np.shape(temp)

(482, 1010)

In [6]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster()
cluster


/group/pawsey0219/ijanekovic/miniconda3/envs/pangeo/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42595 instead
  warnings.warn(
Exception ignored in: <function Cluster.__del__ at 0x2ab4a72f6c10>
Traceback (most recent call last):
  File "/group/pawsey0219/ijanekovic/miniconda3/envs/pangeo/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 107, in __del__
    if self.status != Status.closed:
AttributeError: 'SLURMCluster' object has no attribute 'status'


AttributeError: 'SLURMCluster' object has no attribute 'scheduler_info'

AttributeError: 'SLURMCluster' object has no attribute 'scheduler_info'

SLURMCluster(cores=0, memory=0 B, workers=0/0, jobs=0/0)

In [3]:
cluster = SLURMCluster()

/group/pawsey0219/ijanekovic/miniconda3/envs/pangeo/lib/python3.8/site-packages/distributed/deploy/local.py:127: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  warnings.warn(
/group/pawsey0219/ijanekovic/miniconda3/envs/pangeo/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33101 instead
  warnings.warn(


In [4]:
cluster

AttributeError: 'SLURMCluster' object has no attribute 'scheduler_info'

AttributeError: 'SLURMCluster' object has no attribute 'scheduler_info'

SLURMCluster(cores=0, memory=0 B, workers=0/0, jobs=0/0)

In [5]:
cluster.scale(20)

NotImplementedError: 